<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Breakout_Level_Nearness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0

import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress

# Zaman dilimi ismini döndürme fonksiyonu
def interval_name(interval):
    interval_map = {
        Interval.in_15_minute: '15 Dakika',
        Interval.in_30_minute: '30 Dakika',
        Interval.in_45_minute: '45 Dakika',
        Interval.in_1_hour: '1 Saat',
        Interval.in_2_hour: '2 Saat',
        Interval.in_4_hour: '4 Saat',
        Interval.in_daily: 'Günlük',
        Interval.in_weekly: 'Haftalık',
        Interval.in_monthly: 'Aylık'
    }
    return interval_map.get(interval, 'Günlük')

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimini seçin:")
    for key, (name, _) in intervals.items():
        print(f"{key}. {name}")

    choice = input("Seçiminiz (örnek: 6): ").strip()
    return intervals.get(choice, ('Günlük', Interval.in_daily))

# Düşüş trend çizgisi hesaplama fonksiyonu
def Down_Trend_Line(data, window=5):
    df = data.iloc[:-window].copy()
    hh_pairs = argrelextrema(df['high'].values, comparator=np.greater, order=window)[0]
    highest_point = df.iloc[hh_pairs].nlargest(1, 'high').sort_values(by='datetime')
    max_idx = data.index.get_loc(highest_point.index[0])
    df_next = df.iloc[max_idx:]

    hh_pairs_next = argrelextrema(df_next['high'].values, comparator=np.greater, order=window)[0]
    slopes = []
    intercepts = []

    for hh_next in hh_pairs_next:
        next_hp = df_next.iloc[hh_next]
        slope, intercept, _, _, _ = linregress([highest_point.index[0], next_hp.name], [highest_point['high'].values[0], next_hp['high']])
        slopes.append(slope)
        intercepts.append(intercept)

    slopes = [slope for slope in slopes if slope != 0]
    intercepts = [intercept for slope, intercept in zip(slopes, intercepts) if slope != 0]
    if slopes:
        min_slope_index = np.argmin(np.abs(slopes))
        min_slope = slopes[min_slope_index]
        min_intercept = intercepts[min_slope_index]
        df = data.copy()
        df['trend'] = min_slope * df.index + min_intercept
        df['Entry'] = df['high'] > df['trend']
    return df

# Hisse analizi
tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Zaman dilimi seçimi
chosen_name, chosen_interval = get_interval_choice()
print(f"Seçilen Zaman Dilimi: {chosen_name}")

Titles = ['Hisse Adı', 'Son Fiyat', 'Trend Değeri', 'Yüzde', 'Yakınlık Durumu', 'Kırılma Durumu']
df_down_trend = pd.DataFrame(columns=Titles)

for i in range(len(Hisseler)):
    try:
        order = 5
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=chosen_interval, n_bars=500)
        data = data.reset_index()
        Trend_line = Down_Trend_Line(data, window=order)
        Entry = False
        Signals = Trend_line.tail(order)
        Signals = Signals.reset_index()
        Last_Price = Signals.loc[order-1, 'high']
        Last_Trend = Signals.loc[order-1, 'trend']
        Last_Perc = ((Signals.loc[order-1, 'trend'] - Signals.loc[order-1, 'high']) / Signals.loc[order-1, 'trend']) * 100
        Close_Status = False
        Break_Status = False
        if Last_Perc <= 5 and Last_Trend > Last_Price:
            Close_Status = True

        Entry = Signals['Entry'].any() and not Signals['Entry'].all()
        Last_Entry = Signals.loc[order-1, 'Entry']
        if Entry and Last_Entry:
            Break_Status = True

        L1 = [Hisseler[i], round(Last_Price, 2), round(Last_Trend, 2), round(Last_Perc, 2), Close_Status, Break_Status]
        df_down_trend.loc[len(df_down_trend)] = L1
    except:
        pass

# Yakınlık ve kırılma durumlarını sıralama
df_close = df_down_trend[(df_down_trend['Yakınlık Durumu'] == True)].sort_values(by='Yüzde', ascending=True)
df_Breakout = df_down_trend[(df_down_trend['Kırılma Durumu'] == True)].sort_values(by='Yüzde', ascending=True)

# Sonuçları yazdırma
print('Yakın Olan Hisseler:')
print(df_close.to_string(index=False))
print('\nKırılma Gerçekleşmiş Olanlar:')
print(df_Breakout.to_string(index=False))


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-3tpacodq
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-3tpacodq
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=ef1fe3151880cd1b742a7dac2a5d1fdd633fd073fee7bb98d156d22e9019744a
  Stored in directory: /tmp/pip-ephem-wheel-cache-4whjporr/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00


Lütfen zaman dilimini seçin:
1. 15 Dakika
2. 30 Dakika
3. 45 Dakika
4. 1 Saat
5. 2 Saat
6. 4 Saat
7. Günlük
8. Haftalık
9. Aylık
Seçiminiz (örnek: 6): 7
Seçilen Zaman Dilimi: Günlük
Yakın Olan Hisseler:
Hisse Adı  Son Fiyat  Trend Değeri  Yüzde  Yakınlık Durumu  Kırılma Durumu
    KMPUR      17.61         17.63   0.12             True           False
    THYAO     326.25        326.73   0.15             True           False
    CLEBI    2271.00       2276.35   0.24             True           False
    MOGAN       8.99          9.02   0.30             True           False
    ULKER     119.10        119.82   0.60             True           False
    ECZYT     188.00        189.20   0.64             True           False
    HTTBT      39.42         39.67   0.64             True           False
    TTKOM      50.35         50.74   0.77             True           False
    EMNIS     317.25        319.82   0.80             True           False
    AYCES     442.25        445.94   0.83      